In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logr_clong').getOrCreate()

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
df = sqlContext.sql("SELECT * FROM titanic_csv")
df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
summary| PassengerId| Survived| Pclass| Name| Sex| Age| SibSp| Parch| Ticket| Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
 count| 891| 891| 891| 891| 891| 714| 891| 891| 891| 891| 204| 889|
 mean| 446.0| 0.3838383838383838| 2.308641975308642| null| null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.2042079685746| null| null|
 stddev|257.3538420152301|0.48659245426485753|0.8360712409770491| null| null|14.526497332334035|1.1027434322934315| 0.8060572211299488|471609.26868834975|49.69342859718089| null| null|
 min| 1| 0| 1| Abbing, Mr. Anthony|female| 0.42| 0| 0| 110152| 0.0| A10| C|
 max| 891| 1| 3|van Melkebeke, Mr...| male| 80.0| 8| 6| WE/P 5735| 512.3292| T| S|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+

In [0]:
df.printSchema()

root
-- PassengerId: integer (nullable = true)
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Name: string (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Ticket: string (nullable = true)
-- Fare: double (nullable = true)
-- Cabin: string (nullable = true)
-- Embarked: string (nullable = true)

In [0]:
df.columns

Out[7]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [0]:
myCols = [ 'Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked']

In [0]:
cdf = df.select(myCols)
cdf.printSchema()

root
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Fare: double (nullable = true)
-- Embarked: string (nullable = true)

In [0]:
fdf = cdf.na.drop()
fdf.printSchema()

root
-- Survived: integer (nullable = true)
-- Pclass: integer (nullable = true)
-- Sex: string (nullable = true)
-- Age: double (nullable = true)
-- SibSp: integer (nullable = true)
-- Parch: integer (nullable = true)
-- Fare: double (nullable = true)
-- Embarked: string (nullable = true)

In [0]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [0]:
genderIdxr = StringIndexer(inputCol='Sex', outputCol='SexIndex')
genderEncdr = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [0]:
embarkIdxr = StringIndexer(inputCol='Embarked', outputCol='EmbarkedIndex')
embarkEncdr = OneHotEncoder(inputCol='EmbarkedIndex',outputCol='EmbarkedVec')

In [0]:
ass = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkedIndex','Age','SibSp','Parch','Fare'],outputCol='features')

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
from pyspark.ml import Pipeline

In [0]:
lrt = LogisticRegression(featuresCol='features',labelCol='Survived')

In [0]:
pipeline = Pipeline(stages=[
  genderIdxr,embarkIdxr,
  genderEncdr, embarkEncdr,
  ass, lrt
])

In [0]:
train, test = fdf.randomSplit([.7,.3])

In [0]:
fitted = pipeline.fit(train)

In [0]:
res = fitted.transform(test)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [0]:
res.select('Survived','Prediction').show()

+--------+----------+
Survived|Prediction|
+--------+----------+
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 1.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+--------+----------+
only showing top 20 rows

In [0]:
auc = eval.evaluate(res)

In [0]:
auc

Out[31]: 0.7683701387531756